In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz

In [ ]:
!gunzip green_tripdata_2019-10.csv.gz

## Question 3

In [ ]:
import json

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

## Question 4

In [ ]:
import pandas as pd

GREEN_CASTING_DICT = {
    #"lpep_pickup_datetime": "datetime64[ms]",
    #"lpep_dropoff_datetime": "datetime64[ms]",
    "PULocationID": "Int64",
    "DOLocationID": "Int64",
    "passenger_count": "Int64",
    "trip_distance": "float64",
    "tip_amount": "float64",
}

columns_to_read = [
    "lpep_pickup_datetime",
    "lpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "passenger_count",
    "trip_distance",
    "tip_amount"
]

df = pd.read_csv('green_tripdata_2019-10.csv', usecols=columns_to_read, dtype=GREEN_CASTING_DICT, parse_dates=["lpep_pickup_datetime", "lpep_dropoff_datetime"])
df.head(10)

In [ ]:
from time import time

t0 = time()

for index, row in df.iterrows():
    message = row.to_json()
    producer.send('green-trips', value=message)
producer.flush()

t1 = time()
t1 - t0